In [92]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [93]:
df_train = pd.read_csv('Train.csv')
print(df_train.head())

   Relapse  CEA_Post  Lymphatic  Vascular  pStage  pT  pN  Perineural  Age
0        0       0.0          0         0       4   2   2           0   73
1        0       1.2          1         0       4   2   2           1   73
2        1       1.8          1         0       4   1   2           0   53
3        0       0.0          1         0       4   2   2           0   57
4        1      52.0          1         1       6   4   3           0   65


In [94]:
df_valid = pd.read_csv('Valid.csv')
print(df_valid.head())

   Relapse  CEA_Post  Lymphatic  Vascular  pStage  pT  pN  Perineural  Age
0        0       1.2          1         0       5   1   3           0   42
1        1       1.0          1         0       4   2   2           0   45
2        1       9.2          1         0       6   4   3           0   69
3        1       0.0          0         0       5   5   1           0   64
4        0       2.3          1         1       5   1   3           0   72


In [95]:
df_train.corr()

,Relapse,CEA_Post,Lymphatic,Vascular,pStage,pT,pN,Perineural,Age
Relapse,1.000000,0.163411,0.206385,0.208355,0.180064,0.056499,0.204281,0.130624,-0.001398
CEA_Post,0.163411,1.000000,0.069175,0.056759,0.121155,-0.050621,0.165132,0.201012,-0.000181
Lymphatic,0.206385,0.069175,1.000000,0.228239,0.199909,-0.093983,0.319781,0.160543,-0.076858
Vascular,0.208355,0.056759,0.228239,1.000000,0.129301,0.042075,0.182302,0.250198,0.103310
pStage,0.180064,0.121155,0.199909,0.129301,1.000000,0.370512,0.770096,0.023054,0.099237
pT,0.056499,-0.050621,-0.093983,0.042075,0.370512,1.000000,-0.172943,0.014330,0.166362
pN,0.204281,0.165132,0.319781,0.182302,0.770096,-0.172943,1.000000,0.055978,-0.029191
Perineural,0.130624,0.201012,0.160543,0.250198,0.023054,0.014330,0.055978,1.000000,0.040122
Age,-0.001398,-0.000181,-0.076858,0.103310,0.099237,0.166362,-0.029191,0.040122,1.000000


In [106]:
# InputFeature = df_train[['CEA_Post', 'Lymphatic', 'Vascular', 'pN', 'Perineural']]
Label = df_train[['Relapse']]
# # 딥러닝이니까 Feature Selection을 할 필요가 없네...
# x_val = df_valid[['CEA_Post', 'Lymphatic', 'Vascular', 'pN', 'Perineural']]
y_val = df_valid[['Relapse']]

InputFeature = df_train[df_train.columns[1:]]
x_val = df_valid[df_valid.columns[1:]]

In [97]:
model = Sequential()
model.add(Dense(5, activation='linear', input_shape=(8,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.1), metrics=['accuracy'])

In [98]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 5)                 45        
                                                                 
 dense_13 (Dense)            (None, 1)                 6         
                                                                 
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [99]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
# 원래는 valid로 함. 여기는 train 밖에 없어서 train으로 한 것임
CP = ModelCheckpoint(filepath='-{epoch:03d}-{loss:.4f}-{accuracy:.4f}-{val_loss:.4f}.hdf5',
            monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=4, verbose=1, min_lr=1e-8)

# factor: LR에 곱할 것.
# 0.1-> 0.08 -> 0.064.. epoch 3번 돌았을 때, loss 떨어지면 곱해라!, 1e-8보다는 LR 줄이진 말고.

CALLBACK = [CP, LR]

In [102]:
InputFeature

,Relapse,CEA_Post,Lymphatic,Vascular,pStage,pT,pN,Perineural,Age
1,0,1.2,1,0,4,2,2,1,73
2,1,1.8,1,0,4,1,2,0,53
3,0,0.0,1,0,4,2,2,0,57
4,1,52.0,1,1,6,4,3,0,65
5,1,1.9,1,0,5,5,1,0,77
...,...,...,...,...,...,...,...,...,...
387,0,1.7,0,0,4,3,2,0,57
388,0,4.1,0,0,4,4,1,0,63
389,0,2.2,1,0,5,2,3,0,63
390,0,5.2,0,0,4,1,2,0,57


In [101]:
model.fit(x=InputFeature, y=Label, epochs=100, shuffle=True, batch_size=3, callbacks=CALLBACK, validation_data = (x_val, y_val))

ValueError: Data cardinality is ambiguous:
  x sizes: 391
  y sizes: 392
Make sure all arrays contain the same number of samples.

In [ ]:
model.load_weights('-006-0.9082-0.5816-0.6199.hdf5')

In [ ]:
import scikitplot as skplt
from sklearn.metrics import roc_auc_score

In [ ]:
print(roc_auc_score(y_val, model.predict(x_val)))

In [ ]:
from sklearn import metrics
# ROC curve 시각화

pred = model.predict(x_val)
fpr, tpr, _ = metrics.roc_curve(y_val,  pred)

In [ ]:
#create ROC curve
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()